In [1]:
import numpy as np
import commpy as cp
import scipy.signal as sig
import scipy.linalg as la
import math
import matplotlib.pyplot as plt
from multiprocessing import Pool

import traceback
import warnings
import sys

%matplotlib inline
DEFAULT_SEED = 100
np.random.seed(DEFAULT_SEED)#set the random generator seed to default

In [33]:
def generate_topleitz(row_vector, channel_length):
    first_row = row_vector[:channel_length]
    first_col = row_vector
    return la.toeplitz(first_col, first_row)

def least_squares(A, b):
    return np.linalg.lstsq(A, b)[0]

def perform_least_squares(preamble, preamble_convolved, channel_length):
    A, b = generate_topleitz(preamble, channel_length), preamble_convolved
    prediction = least_squares(A, b)
    return np.array(prediction)

def add_awgn_noise(signal,SNR_dB):
    """  Adds AWGN noise vector to signal 
         to generate a resulting signal vector y of specified SNR in dB
    """
    L=len(signal)
    SNR = 10**(SNR_dB/10.0) #SNR to linear scale
    Esym=np.sum(np.square(np.abs(signal)))/L #Calculate actual symbol energy
    N0=Esym/SNR; #Find the noise spectral density
    if(isinstance(signal[0], complex)):
        noiseSigma=np.sqrt(N0/2.0)#Standard deviation for AWGN Noise when x is complex
        n = noiseSigma*(np.random.randn(1,L)+1j*np.random.randn(1,L))#computed noise 
    else:
        noiseSigma = np.sqrt(N0);#Standard deviation for AWGN Noise when x is real
        n = noiseSigma*np.random.randn(1,L)#computed noise
    y = signal + n #received signal
    
    return y.flatten()

def modulate(data, mod_scheme='BPSK', demod=False):
    """  1. Modulates (or demodulates) data according to the modulation scheme """
    mod_schemes = ['BPSK', 'QPSK']
    data = data.flatten()
    if mod_scheme not in mod_schemes:
        raise ValueError('Unknown modulation scheme, please choose from: '+ ' '.join(mod_schemes))
    elif mod_scheme == 'QPSK':
        modulator = cp.modulation.QAMModem(4)
        if demod:
            return modulator.demodulate(data, "hard")
        return modulator.modulate(data)
    elif mod_scheme == 'BPSK':
        def bpsk_one(x):
            if demod:
                return 0 if x < 0 else 1
            return -1 if x==0 else 1
        bpsk = np.vectorize(bpsk_one)
        return bpsk(data)
    
def apply_channel(signal, channel_function):
    """  2. Convolves signal with channel_function """
    channel_output = sig.convolve(signal, channel_function, mode='same') # convolve input complex data with the channel transfer function
    return channel_output


In [125]:
channel_length = 2
preamble_length = 2000
data_length = 200

modulation_scheme = 'BPSK'

snr = 10

channel_function = np.random.uniform(-1, 1, channel_length) 
channel_function = channel_function / np.linalg.norm(channel_function)

# generate training data
preamble_bits = np.random.randint(0,2, preamble_length) 
preamble_symbols = modulate(preamble_bits, modulation_scheme)
preamble_convolved = apply_channel(preamble_symbols, channel_function)
# generate testing data
data_bits = np.random.randint(0,2, data_length)
data_symbols = modulate(data_bits, modulation_scheme)
data_convolved = add_awgn_noise(apply_channel(data_symbols, channel_function), snr)
print(preamble_symbols.shape, preamble_convolved.shape)
print(preamble_symbols)
print(preamble_convolved)
print(channel_function)
channel_estimate = perform_least_squares(preamble_symbols, preamble_convolved, channel_length)
print('Predicted channel', channel_estimate)

(2000,) (2000,)
[-1  1  1 ... -1  1  1]
[-0.94827725  0.63083395  1.26572056 ... -0.63083395  0.63083395
  1.26572056]
[0.94827725 0.31744331]
Predicted channel [0.94843661 0.31728395]


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  import sys


In [126]:
def generate_A_hat(channel_estimate, data_length):
    matrix = np.zeros((data_length, data_length))
    for i in range(data_length - 1): 
        matrix[i][i] = channel_estimate[0]
        matrix[i+1][i] = channel_estimate[1]
    matrix[data_length - 1][data_length - 1] = channel_estimate[0]
    return matrix

def calc_accuracy(original, predictions):
    return np.sum(original == predictions) / (original.shape[0])


In [128]:
A_hat = generate_A_hat(channel_estimate, data_length)
print(A_hat)
# print('Ahat')
# print(A_hat)
# print('data convolved')
# print(data_convolved)
# print('data convolved 2')
# data_convolved_estimate = np.dot(A_hat, data_symbols)
# print(data_convolved_estimate)
# print('Norm', np.linalg.norm(data_convolved_estimate - data_convolved))
std = 10 ** (-snr / 20)
outer_prod_data = np.outer(data_symbols, data_symbols) #xxT
RHS = np.dot(outer_prod_data, A_hat.T) # xx_TA_T
to_invert = np.dot(np.dot(A_hat, outer_prod_data), A_hat.T) + (std ** 2) * np.identity(A_hat.shape[0]) #A_hat x x.T A_hatT
B = np.dot(RHS, np.linalg.inv(to_invert))
print('RHS')
print(RHS)
print('Inverse')
print(np.linalg.inv(to_invert))
print('Right inverse')
print(right_inv)
# B = np.dot(RHS, np.identity(data_length))
data_est = np.dot(B, data_convolved)
data_symbols_est = np.where(data_est > 0, 1, -1)
print('Data estimate')
print(data_est)
calc_accuracy(data_symbols_est, data_symbols)

[[0.94843661 0.         0.         ... 0.         0.         0.        ]
 [0.31728395 0.94843661 0.         ... 0.         0.         0.        ]
 [0.         0.31728395 0.94843661 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.94843661 0.         0.        ]
 [0.         0.         0.         ... 0.31728395 0.94843661 0.        ]
 [0.         0.         0.         ... 0.         0.31728395 0.94843661]]
RHS
[[ 0.94843661 -0.63115266  0.63115266 ...  1.26572056  1.26572056
   1.26572056]
 [-0.94843661  0.63115266 -0.63115266 ... -1.26572056 -1.26572056
  -1.26572056]
 [ 0.94843661 -0.63115266  0.63115266 ...  1.26572056  1.26572056
   1.26572056]
 ...
 [ 0.94843661 -0.63115266  0.63115266 ...  1.26572056  1.26572056
   1.26572056]
 [ 0.94843661 -0.63115266  0.63115266 ...  1.26572056  1.26572056
   1.26572056]
 [ 0.94843661 -0.63115266  0.63115266 ...  1.26572056  1.26572056
   1.26572056]]
Inverse
[[ 9.9543455   0.03038153 -0.03038153 ... -0.0609274

1.0

In [59]:
print(A_hat.shape)
print(data_symbols.shape)

(200, 200)
(200,)
